# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [2]:
# Or you can use a .env file
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)
os.environ["USER_AGENT"] = "496"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [3]:
import uuid
thread_id = uuid.uuid4()
print(f"Thread ID: {thread_id}")

Thread ID: 844a1f3e-cd19-40dd-a208-d350969e48af


In [4]:
from langsmith import traceable
from anthropic import Anthropic
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

anthropic_client = Anthropic()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the latest question in the conversation.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "user",
            "content": f"{rag_system_prompt}\n\nContext: {formatted_docs}\n\nQuestion: {question}"
        }
    ]
    return call_anthropic(messages)

@traceable(run_type="llm")
def call_anthropic(
    messages: List[dict], model: str = "claude-sonnet-4-5-20250929", temperature: float = 0.0
) -> str:
    return anthropic_client.messages.create(
        model=model,
        max_tokens=1024,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content[0].text

### Now let's run our application twice with this thread_id

In [5]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(
    question,
    langsmith_extra={"metadata": {"thread_id": thread_id}}
)
print(ai_answer)

Based on the documentation, you can add metadata to traces in LangSmith by passing a dictionary of key-value pairs along with your traces. Metadata is useful for storing additional information like the environment, user who initiated the trace, or internal correlation IDs. The documentation mentions that both Python and TypeScript support sending arbitrary metadata and tags with traces, though the specific code example wasn't fully shown in the provided context.


In [6]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(
    question,
    langsmith_extra={"metadata": {"thread_id": thread_id}}
)
print(ai_answer)

Based on the documentation, you can add tags to traces in LangSmith by sending them along with your traces during instrumentation. Tags are strings used to categorize or label a trace, and they're useful for associating additional information like the environment, user, or correlation ID. You can then filter and query traces based on these tags, such as filtering for runs where tags include "experimental" or "beta".


### Let's take a look in LangSmith!